# Training Mask Detection Model

## Load Libraries

In [ ]:
import os
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## Loading Mask and No Mask Images

<pre>
X - Images
Y - Label
    0 - Mask
    1 - No Mask
</pre>

In [ ]:
DATASET = './dataset'
TARGET_SIZE = (224, 224)
images = []
labels = []

# with_mask    : 0
# without_mask : 1

for label, IMAGE_CLASS in enumerate(os.listdir(DATASET)):
    IMAGE_CLASS_FOLDER = os.path.join(DATASET, IMAGE_CLASS)
    for IMAGE in os.listdir(IMAGE_CLASS_FOLDER):
        IMAGE_PATH = os.path.join(IMAGE_CLASS_FOLDER, IMAGE)
        image = load_img(IMAGE_PATH, target_size=TARGET_SIZE)
        image = img_to_array(image)
        image = preprocess_input(image)
        
        images.append(image)
        labels.append(label)
        
images = np.array(images, dtype='float32')
labels = np.array(labels)
 
trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.20, stratify=labels, random_state=42)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


## Image Generator for performing Data Augumentation

In [ ]:
generator = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

## Mask Detection Model built from MOBILENET using Transfer Learning

In [ ]:
Net = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
 
X = Net.output
X = AveragePooling2D(pool_size=(7, 7))(X)
X = Flatten(name="flatten")(X)
X = Dense(128, activation="relu")(X)
X = Dropout(0.5)(X)
X = Dense(1, activation="sigmoid")(X)
 
model = Model(inputs=Net.input, outputs=X)
 
for layer in Net.layers:
    layer.trainable = False
    
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])
 
EPOCHS = 5
BATCH_SIZE = 32
 
_ = model.fit(
    generator.flow(trainX, trainY, batch_size=BATCH_SIZE),
    steps_per_epoch=len(trainX) // BATCH_SIZE,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BATCH_SIZE,
    epochs=EPOCHS)

9412608/9406464 [==============================] - 0s 0us/step
Epoch 1/5
95/95 [==============================] - 64s 339ms/step - loss: 0.3382 - accuracy: 0.8563 - val_loss: 0.1265 - val_accuracy: 0.9765
Epoch 2/5
95/95 [==============================] - 29s 306ms/step - loss: 0.1243 - accuracy: 0.9614 - val_loss: 0.0760 - val_accuracy: 0.9817
Epoch 3/5
95/95 [==============================] - 29s 302ms/step - loss: 0.0749 - accuracy: 0.9796 - val_loss: 0.0565 - val_accuracy: 0.9844
Epoch 4/5
95/95 [==============================] - 29s 306ms/step - loss: 0.0614 - accuracy: 0.9792 - val_loss: 0.0509 - val_accuracy: 0.9804
Epoch 5/5
95/95 [==============================] - 29s 304ms/step - loss: 0.0533 - accuracy: 0.9825 - val_loss: 0.0419 - val_accuracy: 0.9870


## Model Performance

In [ ]:
probs = model.predict(testX, batch_size=BATCH_SIZE)
 
preds = np.where(probs > 0.5, 1, 0)
 
print(classification_report(testY, preds))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       383
           1       0.99      0.98      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



## Saving Trained Mask Detection Model

In [ ]:
model.save("mask_detector.model", save_format="h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
